### 2. Load Data

**[2.1]** Import the function you created `load_sets` from `src/data/sets`

In [1]:
# Solution
from src.data.sets import load_sets

**[2.2]** Load the saved sets from `data/processed`

In [2]:
#Solution:
X_train, y_train, X_val, y_val, X_test, y_test = load_sets(path='../data/processed/')

# 3. Train Xgboost model

**[3.1]** Import the xgboost package as xgb


In [3]:
# Solution:
import xgboost as xgb

**[3.2]** Instantiate the RandomForest class into a variable called rf with random_state=8

In [4]:
# Solution
xgboost1 = xgb.XGBClassifier()

**[3.3]** Task: Fit the model with the prepared data

In [5]:
# Solution
xgboost1.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

**[3.4]** Import `dump` from `joblib` and save the fitted model into the folder `models` as a file called `xgboost_default`

In [6]:
# Solution:
from joblib import dump 

dump(xgboost1,  '../models/xgboost_default.joblib')

['../models/xgboost_default.joblib']

**[3.5]** Save the predictions from this model for the training and validation sets into 2 variables called `y_train_preds` and `y_val_preds`


In [7]:
# Solution:
y_train_preds = xgboost1.predict(X_train)
y_val_preds = xgboost1.predict(X_val)

**[3.6]** Import `print_reg_perf` from `src/models/performance` and display the accuracy and f1 scores of this baseline model on the training and validation sets

In [8]:
# Solution
from src.models.performance import print_class_perf

print_class_perf(y_preds=y_train_preds, y_actuals=y_train, set_name='Training', average='weighted')
print_class_perf(y_preds=y_val_preds, y_actuals=y_val, set_name='Validation', average='weighted')

Accuracy Training: 0.9241850204839105
F1 Training: 0.9239048901039122
Accuracy Validation: 0.9066156852034707
F1 Validation: 0.9061059642094202


#4. Hyperparameter tuning with Hyperopt

**[4.1]** Import Trials, STATUS_OK, tpe, hp, fmin from hyperopt package

In [9]:
# Solution:
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin

**[4.2]** Define the search space for xgboost hyperparameters

In [10]:
space = {
    'max_depth' : hp.choice('max_depth', range(5, 20, 1)),
    'learning_rate' : hp.quniform('learning_rate', 0.01, 0.5, 0.05),
    'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
    'subsample' : hp.quniform('subsample', 0.1, 1, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1.0, 0.05)
}

**[4.3]** Define a function called `objective` with the following logics:
- input parameters: hyperparameter seacrh space (`space`)
- logics: train a xgboost model with the search space and calculate the average accuracy score for cross validation with 10 folds
- output parameters: dictionary with the loss score and STATUS_OK

In [11]:
# Solution:
def objective(space):
    from sklearn.model_selection import cross_val_score
    
    xgboost = xgb.XGBClassifier(
        max_depth = int(space['max_depth']),
        learning_rate = space['learning_rate'],
        min_child_weight = space['min_child_weight'],
        subsample = space['subsample'],
        colsample_bytree = space['colsample_bytree']
    )
    
    acc = cross_val_score(xgboost, X_train, y_train, cv=10, scoring="accuracy").mean()

    return{'loss': 1-acc, 'status': STATUS_OK }

**[4.4]** Launch Hyperopt search and save the result in a variable called `best`

In [ ]:
best = fmin(
    fn=objective,   
    space=space,       
    algo=tpe.suggest,       
    max_evals=5
)

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

**[4.5]** Print the best set of hyperparameters

In [ ]:
# Solution:
print("Best: ", best)

**[4.6]** Instantiate a XGBClassifier with best set of hyperparameters

In [ ]:
# Solution:
xgboost2 = xgb.XGBClassifier(
    max_depth = best['max_depth'],
    learning_rate = best['learning_rate'],
    min_child_weight = best['min_child_weight'],
    subsample = best['subsample'],
    colsample_bytree = best['colsample_bytree']
)

**[4.7]** Fit the model with the prepared data

In [ ]:
# Solution:
xgboost2.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.35000000000000003, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.15000000000000002, max_delta_step=0, max_depth=3,
              min_child_weight=5.0, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.9500000000000001,
              tree_method='exact', validate_parameters=1, verbosity=None)

**[4.8]** Display the accuracy and f1 scores of this baseline model on the training and validation sets

In [ ]:
# Solution:
print_class_perf(y_preds=xgboost2.predict(X_train), y_actuals=y_train, set_name='Training', average='weighted')
print_class_perf(y_preds=xgboost2.predict(X_val), y_actuals=y_val, set_name='Validation', average='weighted')

Accuracy Training: 0.8756679155432613
F1 Training: 0.8741427824032268
Accuracy Validation: 0.8759217630622492
F1 Validation: 0.8742533580742383


**[4.9]** Save the fitted model into the folder models as a file called `xgboost_best`

In [21]:
# Solution:
dump(xgboost2,  '../models/xgboost_best.joblib')

['../models/xgboost_best.joblib']